Preprocess data

In [1]:
import pandas as pd
from tqdm import tqdm
import csv

In [2]:
brandbankExtract = r'Data\LatestProducts.csv'

In [3]:
df = pd.read_csv(brandbankExtract)
df=df.fillna('')
df = df[df['isDelete'] != 1]
df['text'] = df[[
    'description',
    'allergyAdvice',
    'recycling',
    'recycling_other',
    'brands',
    'marketing',
    'ingredients',
    'features',
    'storage',
    'preserves']].agg(' '.join, axis=1)
df=df.drop(['name','gtin','isDelete'
    ,'pl2','pl3',
    'description',
    'allergyAdvice',
    'recycling',
    'recycling_other',
    'brands',
    'marketing',
    'ingredients',
    'features',
    'storage',
    'preserves'],axis=1)
df.rename({'pl1':'category'},axis=1, inplace=True)
df.drop(df[df['category'] =='Kruidenierswaren'].index, inplace=True)#onyl 1 example of this so dropping for simplicity
df['category'] = df['category'].str.replace(" ", "_")
#df = df[df['category'] != '']
df = df[df['category'] != 'Unallocated']
df['text'] = df['text'].str.lower()
df.head()

,category,text
0,Fruit_&_Vegetables,essential waitrose leeks loose waitrose
1,Fruit_&_Vegetables,loose peach unbranded
2,Fruit_&_Vegetables,asda sweet potato asda
4,Deli,wr gateaux each
6,Grocery,waitrose oatmeal and sultana cookie waitros...


RAKE_NLTK

In [4]:
from rake_nltk import Rake
import nltk

In [5]:
#nltk.download('stopwords') #Only needs to be run once to download stop words

In [6]:
def get_num_words(row):
    return len(str(row['Phrase']).split())

def get_perc_phrase(row,total):
    return (float(row['Score'])/total*100)

In [7]:
def get_Rake_phrases(df):
    r = Rake()
    r.extract_keywords_from_sentences(df['text'])
    phrases=r.get_ranked_phrases_with_scores()
    df = pd.DataFrame(phrases, columns = ['Rank', 'Phrase'])
    df['Phrase'] = df['Phrase'].str.strip()
    phrase_counts = df['Phrase'].value_counts()
    phrase_counts = phrase_counts.reset_index()
    phrase_counts.columns=['Phrase','Score']
    phrase_counts['Length'] = phrase_counts.apply (lambda row: get_num_words(row), axis=1)
    phrase_counts=phrase_counts.loc[phrase_counts['Length'] <=6]
    total = float(phrase_counts['Score'].sum())
    phrase_counts['Perc'] = phrase_counts.apply (lambda row: get_perc_phrase(row,total), axis=1)
    return phrase_counts

In [8]:
df_raked = get_Rake_phrases(df)
#phrase_counts.to_csv(r'C:\Users\Clamfighter\Documents\GitHub\NLP\Impact Score Key Phrase\Extracted Phrases\Rake NLTK\Brandbank Latest Phrase Extraction.csv',index = False)

In [9]:
df_raked

,Phrase,Score,Length,Perc
0,contains,7928,1,1.076919
1,may contain,5579,2,0.757837
2,suitable,5158,1,0.700650
3,salt,4711,1,0.639930
4,sugar,4652,1,0.631916
...,...,...,...,...
172741,"1 %), dried herbs",1,4,0.000136
172742,"1 %), skimmed cows",1,4,0.000136
172743,"parsley ), natural flavourings",1,4,0.000136
172744,tasty honey mustard sauce,1,4,0.000136


In [10]:
phrase_count=[]

df_phraseCount = df_raked.loc[df_raked['Score'] != 1] #filters Score of 1 out. will be added back in later
phrases = df_phraseCount['Phrase']
text = df['text']
for phrase in tqdm(phrases): #find number of phrases for all strings
    phrase_count.append(sum(phrase in s for s in text))
phrase_count

 10%|▉         | 5088/51606 [01:27<12:53, 60.16it/s]

In [ ]:
df_phraseCount['Count'] = phrase_count
df_phraseCount.drop(['Score','Length','Perc'],axis=1, inplace=True)
df_out = df_raked.merge(df_phraseCount, on='Phrase', how='left')
df_out.fillna(1,inplace=True)
df_out

C:\Users\admin_1878\AppData\Local\Temp\ipykernel_11008\2573982755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phraseCount['Count'] = phrase_count
C:\Users\admin_1878\AppData\Local\Temp\ipykernel_11008\2573982755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phraseCount.drop(['Score','Length','Perc'],axis=1, inplace=True)


,Phrase,Score,Length,Perc,Count
0,contains,46090,1,1.043119,46566.0
1,free,27964,1,0.632888,28091.0
2,may contain,27856,2,0.630443,14558.0
3,suitable,26888,1,0.608535,32327.0
4,sugar,25511,1,0.577371,33710.0
...,...,...,...,...,...
500458,best fighting units,1,3,0.000023,1.0
500459,piers morgan unbranded piers morgan,1,5,0.000023,1.0
500460,extremely perilous christmas,1,3,0.000023,1.0
500461,trifles unbranded,1,2,0.000023,1.0


In [ ]:
#df_out.to_csv(r'out_lower.csv',index = False)

NameError: name 'df_out' is not defined